In [17]:
import numpy as np
import string
import re
import random
import textwrap

In [2]:
letter_1 = list(string.ascii_lowercase)
letter_2 = list(string.ascii_lowercase)

true_mapping = {}
reverse_mapping = {}

random.shuffle(letter_2)
for k,v in zip(letter_1, letter_2):
    true_mapping[k] = v
    reverse_mapping[v] = k  ## Just for Sanity Check

In [3]:
pie = np.zeros(26)
A = np.ones((26, 26))
def update_transition(ch0, ch1):
    i = ord(ch0) - 97
    j = ord(ch1) - 97
    A[i,j] += 1

def update_pie(ch0):
    i = ord(ch0) - 97
    pie[i] += 1

def word_prob(word):
    i = ord(word[0]) - 97
    logp = np.log(pie[i])
    for ch in word[1:]:
        j = ord(ch) - 97
        logp += np.log(A[i,j])
        i = j
    return logp

def string_prob(words):
    if type(words)==str:
        words = words.split()
    logp = 0
    for word in words:
        logp += word_prob(word)
    return logp

In [4]:
regex = re.compile('[^a-zA-Z]')
for line in open('/Users/lovedeepsingh/Documents/Natural-Language-Processing/Cipher Decrypter/Mobidick.txt'):
    line = line.lower().rstrip()
    if line:
        line = regex.sub(' ', line)
        tokens = line.split()
        for token in tokens:
            ch0 = token[0]
            update_pie(ch0)
            for ch1 in token[1:]:
                update_transition(ch0, ch1)
                ch0 = ch1
pie /= pie.sum()
A /= A.sum(axis=1, keepdims=True) 

In [5]:
with open('/Users/lovedeepsingh/Documents/Natural-Language-Processing/Cipher Decrypter/Sample_text.txt') as f:
    sample_message = f.readlines()
sample_message = ''.join(sample_message)

In [6]:
def encode(lines):
    lines = lines.lower()
    tokens = regex.sub(' ', lines)
    new_message = []
    for token in tokens:
        coded_words = token
        if token in true_mapping:
            coded_words = true_mapping[token]
        new_message.append(coded_words)
    return (''.join(new_message))

In [7]:
encoded_message = encode(sample_message)

In [8]:
encoded_message

'o bpga uteacgf ftha bpg dbwggb xaf mteaf  xd o gnzglbgf  bpxb bpgwg hxd x qghd oa x uxag hpolp wead ftha ks tag hxuu tm bpg cxwfga  o ugab bpg tdbugwd x pxaf oa wekkoac ftha bpgow ptwdgd  xaf wglgoygf oa gnlpxacg bhtzgalg  x cuxdd tm pxum xaf pxum  bht mouud tm dpxc btkxllt  xaf xd qelp oamtwqxbota xd o lteuf fgdowg xkteb qodd xfugw  bt dxs atbpoac tm pxum x ftjga tbpgw zgtzug oa bpg agocpktewpttf oa hptq o hxd atb oa bpg ugxdb oabgwgdbgf  keb hptdg kotcwxzpogd o hxd ltqzguugf bt uodbga bt '

In [9]:
def decode(lines, mapping):
    new_message = []
    for token in lines:
        coded_words = token
        if token in mapping:
            coded_words = mapping[token]
        new_message.append(coded_words)
    return (''.join(new_message))

In [10]:
decoding_sample_test = decode(encoded_message, reverse_mapping)

In [11]:
decoding_sample_test

'i then lounged down the street and found  as i expected  that there was a mews in a lane which runs down by one wall of the garden  i lent the ostlers a hand in rubbing down their horses  and received in exchange twopence  a glass of half and half  two fills of shag tobacco  and as much information as i could desire about miss adler  to say nothing of half a dozen other people in the neighbourhood in whom i was not in the least interested  but whose biographies i was compelled to listen to '

In [12]:
dna_pool = []
for _ in range(20):
    dna = list(string.ascii_lowercase)
    random.shuffle(dna)
    dna_pool.append(dna)

In [13]:
def evolve_offspring(dna_pool, n_childern):
    offspring = []
    for dna in dna_pool:
        for _ in range(n_childern):
            copy = dna.copy()
            i = np.random.randint(len(copy))
            j = np.random.randint(len(copy))
            temp = copy[i]
            copy[i] = copy[j]
            copy[j] = temp
            offspring.append(copy)
    return offspring + dna_pool

In [14]:
n_iters = 1000
best_dna = None
best_mapping = None
scores = np.zeros(n_iters)
best_score = float('-inf')
for i in range(n_iters):
    if i>0:
        dna_pool = evolve_offspring(dna_pool, 3)
    dna_score = {}
    for dna in dna_pool:
        current_mapping = {}
        for k,v in zip(letter_1, dna):
            current_mapping[k] = v
        decoded_msg = decode(encoded_message, current_mapping)
        score = string_prob(decoded_msg)
        dna_score[''.join(dna)] = score

        if score > best_score:
            best_score = score
            best_dna = dna
            best_mapping = current_mapping

    scores[i] = np.mean(list(dna_score.values()))
    sorted_data = sorted(dna_score.items(), key=lambda x:x[1], reverse=True)
    dna_pool = [list(k) for k,v in sorted_data[:5]]
    if i % 200 == 0:
        print('iter: ', i, 'Current Score: ', scores[i], 'Best Score So far: ', best_score)

itter:  0 Current Score:  -2028.5614750695458 Best Score So far:  -1760.1538173658453
itter:  200 Current Score:  -1066.489055127954 Best Score So far:  -977.5203393818056
itter:  400 Current Score:  -1028.6369666108603 Best Score So far:  -941.3533967052797
itter:  600 Current Score:  -1026.6242894973823 Best Score So far:  -941.3533967052797
itter:  800 Current Score:  -1063.2707769084432 Best Score So far:  -941.3533967052797


In [16]:
best_decoded = decode(encoded_message, best_mapping)
print('Log-likelihood of real message is :', string_prob(decoding_sample_test))
print('Log-likelihood of our calculated message is :', string_prob(regex.sub(' ',best_decoded)))

Log-likelihood of real message is : -933.1168491858226
Log-likelihood of our calculated message is : -941.3533967052797


In [18]:
print('Our Message:\n', textwrap.fill(best_decoded))
print('Real Message:\n', decoding_sample_test)

Our Message:
 i then lounged down the street and yound  as i expected  that there
was a mews in a lane which runs down fb one wall oy the garden  i lent
the ostlers a hand in ruffing down their horses  and received in
exchange twopence  a glass oy haly and haly  two yills oy shag tofacco
and as much inyormation as i could desire afout miss adler  to sab
nothing oy haly a doken other people in the neighfourhood in whom i
was not in the least interested  fut whose fiographies i was compelled
to listen to
Real Message:
 i then lounged down the street and found  as i expected  that there was a mews in a lane which runs down by one wall of the garden  i lent the ostlers a hand in rubbing down their horses  and received in exchange twopence  a glass of half and half  two fills of shag tobacco  and as much information as i could desire about miss adler  to say nothing of half a dozen other people in the neighbourhood in whom i was not in the least interested  but whose biographies i was compe